# API Test Notebook

This notebook provides a function to test LLM API calls using log parameters.

In [9]:
import base64
import datetime
import logging
import time
import json
import csv
import os
from collections import namedtuple
from io import BytesIO

import google.generativeai as genai
from anthropic import Anthropic
from google.generativeai import caching
from openai import OpenAI
from balrog.client import OpenAIWrapper, LLMResponse

In [11]:
# Define the test_api_call function
def test_api_call(log_data):
    """Test function to manually call the API using log parameters.
    
    Args:
        log_data (dict): Dictionary containing the API call parameters from logs.
            Expected format:
            {
                "messages": list of message dicts,
                "model": str,
                "temperature": float
            }
    
    Returns:
        LLMResponse: The response from the API.
    """
    # Create a mock client config
    class MockClientConfig:
        def __init__(self, model_id, temperature):
            self.client_name = "openai"  # Default to OpenAI
            self.model_id = model_id
            self.base_url = None
            self.timeout = 30
            self.generate_kwargs = {"temperature": temperature}
            self.max_retries = 3
            self.delay = 1
            self.alternate_roles = False

    # Create client config from log data
    client_config = MockClientConfig(
        model_id=log_data["model"],
        temperature=log_data.get("temperature", 1.0)
    )
    
    # Create the appropriate client
    client = OpenAIWrapper(client_config)
    
    # Convert messages to the format expected by the client
    class Message:
        def __init__(self, role, content, attachment=None):
            self.role = role
            self.content = content
            self.attachment = attachment
    
    messages = [
        Message(msg["role"], msg["content"])
        for msg in log_data["messages"]
    ]
    
    # Make the API call
    try:
        response = client.generate(messages)
        print(f"API Call Successful!")
        print(f"Model: {response.model_id}")
        print(f"Completion: {response.completion}")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Input Tokens: {response.input_tokens}")
        print(f"Output Tokens: {response.output_tokens}")
        return response
    except Exception as e:
        print(f"API Call Failed: {str(e)}")
        raise

## Example Usage

Here's how to use the test_api_call function with your log data:

In [12]:
# Example log data
log_data = {
    "messages": [
        {
            "role": "user",
            "content": "You are an agent playing TextWorld, a text-based adventure game where you are in a randomly generated\n    maze and must find the coin. You need to explore different rooms to find the target object.\n    Here are the available commands: goal: print the goal of this game go <dir>: move the player north, east,\n    south, or west. You can only go in the direction indicated with something like an exit or a door. take coin:\n    2in the game by 'take coin' if you see the coin in the room\n    The only action you can do is go <dir> to explore the maze and 'take coin' when you see the coin in the\n    room.\n    You have 80 steps to complete the task. Restarting is forbidden.Observation:\n-= Bathroom =-\nYou've entered a bathroom. I guess you better just go and list everything you see here.\n\n\n\nYou need an unguarded exit? You should try going east. There is an exit to the north. Don't worry, it is unguarded. There is an exit to the west. Don't worry, it is unguarded."
        },
        {
            "role": "assistant",
            "content": "Previous plan:\n```\n<scratch-pad>\nMAP:\n- Bathroom: exits N? E? W?, visited=True\nPATH: [Bathroom]\nFirst unexplored exit in Bathroom: N\n</scratch-pad>\n<|ACTION|>go north<|END|>"
        },
        {
            "role": "user",
            "content": "Observation:\n-= Salon =-\nYou've seen better salons, but at least this one seems pretty typical. You begin to take stock of what's in the room.\n\n\n\nThere is an unblocked exit to the east. There is an exit to the north. Don't worry, it is unblocked. There is an unguarded exit to the south. You don't like doors? Why not try going west, that entranceway is unguarded."
        }
    ],
    "model": "deepseek-reasoner",
    "temperature": 1
}

# Make the API call
response = test_api_call(log_data)

API Call Failed: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Important Notes

1. Make sure you have the necessary API keys set up in your environment variables:
   - For OpenAI models: Set `OPENAI_API_KEY`
   - For Claude models: Set `ANTHROPIC_API_KEY`
   - For Gemini models: Set `GOOGLE_API_KEY`

2. The notebook includes retry logic (3 retries with exponential backoff) in case of temporary failures

3. You can modify the log_data dictionary to test different scenarios